In [88]:
#from google.colab import drive

# Mount Google Drive
#drive.mount('/content/drive')

In [89]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np

In [90]:
# Example dataset
sentences = ["No me gusta el fútbol", "Nunca he estado tan feliz"]
tags = [["NEG", "O", "O", "O", "O"], ["NEG", "O", "O", "O", "O"]]

# Create vocabulary and tag mappings
word_vocab = {"<PAD>": 0, "No": 1, "me": 2, "gusta": 3, "el": 4, "fútbol": 5, "Nunca": 6, "he": 7, "estado": 8, "tan": 9, "feliz": 10}
tag_vocab = {"O": 0, "NEG": 1}
word_vocab_size = len(word_vocab)
tag_vocab_size = len(tag_vocab)

# Preprocessing function to encode sentences and tags
def encode_sentence(sentence, word_vocab):
    return [word_vocab[word] for word in sentence.split()]

def encode_tags(tags, tag_vocab):
    return [tag_vocab[tag] for tag in tags]

encoded_sentences = [encode_sentence(sentence, word_vocab) for sentence in sentences]
encoded_tags = [encode_tags(tag, tag_vocab) for tag in tags]

# Padding
max_len = max(len(s) for s in encoded_sentences)
padded_sentences = [s + [0] * (max_len - len(s)) for s in encoded_sentences]
padded_tags = [t + [0] * (max_len - len(t)) for t in encoded_tags]

# Convert to tensors
X = torch.tensor(padded_sentences, dtype=torch.float32)
y = torch.tensor(padded_tags, dtype=torch.float32)

# Dataset and DataLoader
class NegationDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

dataset = NegationDataset(X, y)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [91]:
# Model definition
class NegationDetectionModel(nn.Module):
    def __init__(self, word_embedding_dim, hidden_dim, num_layers, output_dim):
        super(NegationDetectionModel, self).__init__()
        
        # BiLSTM Layer
        self.bilstm = nn.LSTM(word_embedding_dim, hidden_dim, num_layers, bidirectional=True, batch_first=True)
        
        # Dense Layer
        self.fc = nn.Linear(hidden_dim * 2, output_dim) # hidden_dim * 2 is done because is BIdirectional. Hence, we have the double dimensions
        
    def forward(self, word_embeds):
        
        # BiLSTM
        lstm_out, _ = self.bilstm(word_embeds)
        # Dense Layer
        out = self.fc(lstm_out)
        
        return out

# Hyperparameters
word_embedding_dim = 5 # 300 word, 300 lemma, 11 pos
hidden_dim = 5
output_dim = word_embedding_dim
num_layers=3
# Instantiate the model
model = NegationDetectionModel(word_embedding_dim, hidden_dim, num_layers, output_dim)


In [92]:
# Training function
def train(model, criterion, optimizer):
    # Training loop
    for epoch in range(10):
        for sequences, targets in dataloader:
            # Forward pass
            outputs = model(sequences)
            print(outputs, targets)
            # Compute loss
            loss = criterion(outputs, targets)
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

In [93]:
# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train(model, criterion, optimizer)

tensor([[ 0.1396, -0.3011,  0.3598, -0.3008, -0.0302],
        [ 0.1276, -0.2851,  0.3378, -0.2934,  0.0175]],
       grad_fn=<AddmmBackward0>) tensor([[1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.]])
Epoch 1, Loss: 1.4844579696655273
tensor([[ 0.1484, -0.2982,  0.3531, -0.3050, -0.0341],
        [ 0.1394, -0.2819,  0.3448, -0.2955,  0.0080]],
       grad_fn=<AddmmBackward0>) tensor([[1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.]])
Epoch 2, Loss: 1.475182294845581
tensor([[ 0.1504, -0.3019,  0.3550, -0.3056, -0.0381],
        [ 0.1368, -0.2867,  0.3339, -0.2982,  0.0133]],
       grad_fn=<AddmmBackward0>) tensor([[1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.]])
Epoch 3, Loss: 1.4734771251678467
tensor([[ 0.1590, -0.2990,  0.3486, -0.3095, -0.0421],
        [ 0.1489, -0.2834,  0.3410, -0.3004,  0.0039]],
       grad_fn=<AddmmBackward0>) tensor([[1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.]])
Epoch 4, Loss: 1.4643020629882812
tensor([[ 0.1611, -0.3025,  0.3503, -0.3103, 

In [96]:
# Test the model
def test_model(model, test_loader, criterion):

    model.eval()  # Set model to evaluation mode

    test_loss = 0.0

    with torch.no_grad():
        for inputs in test_loader:
            #inputs = inputs.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, inputs)

            test_loss += loss.item()

    avg_test_loss = test_loss / len(test_loader)
    print(f"Average Test Loss: {avg_test_loss}")
# Test prediction
test_sentence = "Nunca he estado tan feliz"
predicted_tags = predict(model, test_sentence)
print("Sentence:", test_sentence)
print("Predicted Tags:", predicted_tags)

Sentence: Nunca he estado tan feliz
Predicted Tags: tensor([[ 0.1679, -0.2771,  0.3260, -0.3186, -0.0201]])


In [ ]:
# Save the model state dictionary to Google Drive
#torch.save(model.state_dict(), '/content/drive/MyDrive/oscar/biLSTM.pt')